In [1]:
!pip install numpy pandas matplotlib seaborn
!pip install scikit-learn
!pip install jinja2
!pip install sentence-transformers hdbscan

# 씨마켓플레이스

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import rcParams
from matplotlib import font_manager as fm

# 폰트 경로 직접 지정
font_path = '/System/Library/Fonts/Supplemental/AppleGothic.ttf'  # Mac 기본 폰트
font_prop = fm.FontProperties(fname=font_path)
# (, fontproperties=font_prop) 직접 추가해야 한글화 
# rcParams에 적용
rcParams['font.family'] = font_prop.get_name()
rcParams['axes.unicode_minus'] = False



# 데이터 로드

In [3]:
df = pd.read_csv('/Users/c-market/Downloads/data_list.csv', low_memory=False)

df.columns = df.columns.str.replace(' ', '_')

In [4]:
df

,NO,ROWNUM,totalCount,pdt_cd,pdt_share,pdt_share_txt,send_chk,send_chk_txt,pdt_code,pdt_name,...,pdt_cas,pdt_price,pdt_request_date,pdt_simg_type,pdt_simg,pdt_simg_org,pdt_simg_size,brand_name,dam_name,pdt_simg_file_key
0,1,104488,104488,100001,Y,공유,Y,전송완료,SO,전산기록지(80컬럼/양미싱),...,NaN,25600,50:34.9,U,NaN,NaN,NaN,오피스디포,윤갑석,NaN
1,2,104487,104488,100002,Y,공유,Y,전송완료,SO,전산기록지(132컬럼/양미싱),...,NaN,36100,50:34.9,U,NaN,NaN,NaN,오피스디포,윤갑석,NaN
2,3,104486,104488,100003,Y,공유,Y,전송완료,SO,팩스용지(15mX210mm/한솔),...,NaN,2100,50:34.9,U,NaN,NaN,NaN,오피스디포,윤갑석,NaN
3,4,104485,104488,100004,Y,공유,Y,전송완료,SO,팩스용지(15mX216mm/한솔),...,NaN,2100,50:34.9,U,NaN,NaN,NaN,오피스디포,윤갑석,NaN
4,5,104484,104488,100005,Y,공유,Y,전송완료,SO,팩스용지(30mX210mm/한솔),...,NaN,3600,50:34.9,U,NaN,NaN,NaN,오피스디포,윤갑석,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104483,104484,5,104488,204575,Y,공유,Y,전송완료,SZ,BKA-001 무릎 보호대 K,...,NaN,0,25:45.8,NaN,cmarket_empty.png,NaN,NaN,NaN,백제현,NaN
104484,104485,4,104488,204574,Y,공유,Y,전송완료,SZ,BKH-002 손목 보호대,...,NaN,0,25:45.8,NaN,cmarket_empty.png,NaN,NaN,NaN,백제현,NaN
104485,104486,3,104488,204585,Y,공유,Y,전송완료,SZ,[고려에이스] Centrifuge Tube Rack 스텐원심관랙,...,NaN,0,31:18.6,NaN,cmarket_empty.png,NaN,NaN,고려에이스,박예린,NaN
104486,104487,2,104488,204587,Y,공유,Y,전송완료,SZ,Bambu 저온용 재사용 가능 스풀,...,NaN,0,52:19.6,NaN,cmarket_empty.png,NaN,NaN,뱀부랩,안재용,NaN


In [5]:
def summary(df):
    summry = pd.DataFrame(df.dtypes, columns=['data type'])
    summry['#missing'] = df.isnull().sum().values
    summry['Duplicate'] = df.duplicated().sum()
    summry['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summry['min'] = desc['min'].values
    summry['max'] = desc['max'].values
    summry['avg'] = desc['mean'].values
    summry['std dev'] = desc['std'].values
    summry['top value'] = desc['top'].values
    summry['Freq'] = desc['freq'].values

    return summry

In [6]:
summary(df).style.set_caption("**Summary of the product Data**").\
background_gradient(cmap='Pastel2_r', axis=0). \
set_properties(**{'border': '1.3px dotted', 'color': '', 'caption-side': 'left'})

,data type,#missing,Duplicate,#unique,min,max,avg,std dev,top value,Freq
NO,int64,0,0,104488,1.000000,104488.000000,52244.500000,30163.231801,nan,nan
ROWNUM,object,0,0,104487,nan,nan,nan,nan,0:00,2
totalCount,object,0,0,4,nan,nan,nan,nan,104488,104451
pdt_cd,object,0,0,104488,nan,nan,nan,nan,100001,1
pdt_share,object,0,0,1,nan,nan,nan,nan,Y,104488
pdt_share_txt,object,0,0,1,nan,nan,nan,nan,공유,104488
send_chk,object,0,0,2,nan,nan,nan,nan,Y,104466
send_chk_txt,object,0,0,2,nan,nan,nan,nan,전송완료,104466
pdt_code,object,3,0,76,nan,nan,nan,nan,SZ,46466
pdt_name,object,1,0,101448,nan,nan,nan,nan,삭제,98


# 인사이트 
spec부분에 상세규격 별도기재 부분에 대한 해석을 논의해보아야함.     
104488개 중에 101448개의 이름이 일치하지 않음. < 분류가 필요한 이유     
pdt_name에 brand 이름이 들어간 경우를 처리해야하는데 어떻게 할지. 우선 괄호를 다 지우고 단어들을 토큰화 시킬지.

전처리 목록 - 소문자 통일, 괄호,역슬래시 제거하고 공백 넣어서 토큰화 편리하게 만들기, 앞뒤 공백 제거, 


In [7]:
# 원본 데이터는 df
df_test = df.copy()

import re

def preprocess_pdt_name(name):
    name = str(name).lower()  # object → 문자열 + 소문자
    # 괄호, 역슬래시, 슬래시 제거하고 위치에 공백 한 칸 추가
    name = re.sub(r'[()/\\]', ' ', name)
    # 연속된 공백을 하나로 통일
    name = re.sub(r'\s+', ' ', name)
    return name.strip()  # 앞뒤 공백 제거

# df_test에 적용
df_test['pdt_name'] = df_test['pdt_name'].apply(preprocess_pdt_name)

# 확인
print(df_test[['pdt_name']].head(10))



                        pdt_name
0                 전산기록지 80컬럼 양미싱
1                전산기록지 132컬럼 양미싱
2              팩스용지 15mx210mm 한솔
3              팩스용지 15mx216mm 한솔
4              팩스용지 30mx210mm 한솔
5              팩스용지 30mx216mm 한솔
6     복사용지a4 80g 더블에이 500매x5권 박스
7  컬러레이저용지a4 90g 500매 neusiedler
8  컬러레이저용지a3 90g 500매 neusiedler
9     복사용지b4 80g 더블에이 500매x5권 박스


In [8]:
# 공백 기준 토큰화
df_test['tokens'] = df_test['pdt_name'].apply(lambda x: x.split())

# 확인
print(df_test[['pdt_name','tokens']].head(10))


                        pdt_name                              tokens
0                 전산기록지 80컬럼 양미싱                  [전산기록지, 80컬럼, 양미싱]
1                전산기록지 132컬럼 양미싱                 [전산기록지, 132컬럼, 양미싱]
2              팩스용지 15mx210mm 한솔               [팩스용지, 15mx210mm, 한솔]
3              팩스용지 15mx216mm 한솔               [팩스용지, 15mx216mm, 한솔]
4              팩스용지 30mx210mm 한솔               [팩스용지, 30mx210mm, 한솔]
5              팩스용지 30mx216mm 한솔               [팩스용지, 30mx216mm, 한솔]
6     복사용지a4 80g 더블에이 500매x5권 박스    [복사용지a4, 80g, 더블에이, 500매x5권, 박스]
7  컬러레이저용지a4 90g 500매 neusiedler  [컬러레이저용지a4, 90g, 500매, neusiedler]
8  컬러레이저용지a3 90g 500매 neusiedler  [컬러레이저용지a3, 90g, 500매, neusiedler]
9     복사용지b4 80g 더블에이 500매x5권 박스    [복사용지b4, 80g, 더블에이, 500매x5권, 박스]


In [9]:
# 공백 기준 토큰화
df_test['tokens'] = df_test['pdt_name'].apply(lambda x: x.split())

# 확인
print(df_test[['pdt_name','tokens']].head(10))


                        pdt_name                              tokens
0                 전산기록지 80컬럼 양미싱                  [전산기록지, 80컬럼, 양미싱]
1                전산기록지 132컬럼 양미싱                 [전산기록지, 132컬럼, 양미싱]
2              팩스용지 15mx210mm 한솔               [팩스용지, 15mx210mm, 한솔]
3              팩스용지 15mx216mm 한솔               [팩스용지, 15mx216mm, 한솔]
4              팩스용지 30mx210mm 한솔               [팩스용지, 30mx210mm, 한솔]
5              팩스용지 30mx216mm 한솔               [팩스용지, 30mx216mm, 한솔]
6     복사용지a4 80g 더블에이 500매x5권 박스    [복사용지a4, 80g, 더블에이, 500매x5권, 박스]
7  컬러레이저용지a4 90g 500매 neusiedler  [컬러레이저용지a4, 90g, 500매, neusiedler]
8  컬러레이저용지a3 90g 500매 neusiedler  [컬러레이저용지a3, 90g, 500매, neusiedler]
9     복사용지b4 80g 더블에이 500매x5권 박스    [복사용지b4, 80g, 더블에이, 500매x5권, 박스]


In [10]:
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# 원본 데이터 df에서 테스트용 복사본 생성
df_test2 = df.copy()

def preprocess_pdt_name(name):
    name = str(name).lower()            # 소문자
    name = re.sub(r'[()/\\\[\]]', ' ', name)
  # 괄호, 슬래시, 역슬래시 제거 → 공백
    name = re.sub(r'\s+', ' ', name)    # 연속 공백을 한 칸으로
    return name.strip()                  # 앞뒤 공백 제거

df_test2['pdt_name_clean'] = df_test2['pdt_name'].apply(preprocess_pdt_name)

df_test2

,NO,ROWNUM,totalCount,pdt_cd,pdt_share,pdt_share_txt,send_chk,send_chk_txt,pdt_code,pdt_name,...,pdt_price,pdt_request_date,pdt_simg_type,pdt_simg,pdt_simg_org,pdt_simg_size,brand_name,dam_name,pdt_simg_file_key,pdt_name_clean
0,1,104488,104488,100001,Y,공유,Y,전송완료,SO,전산기록지(80컬럼/양미싱),...,25600,50:34.9,U,NaN,NaN,NaN,오피스디포,윤갑석,NaN,전산기록지 80컬럼 양미싱
1,2,104487,104488,100002,Y,공유,Y,전송완료,SO,전산기록지(132컬럼/양미싱),...,36100,50:34.9,U,NaN,NaN,NaN,오피스디포,윤갑석,NaN,전산기록지 132컬럼 양미싱
2,3,104486,104488,100003,Y,공유,Y,전송완료,SO,팩스용지(15mX210mm/한솔),...,2100,50:34.9,U,NaN,NaN,NaN,오피스디포,윤갑석,NaN,팩스용지 15mx210mm 한솔
3,4,104485,104488,100004,Y,공유,Y,전송완료,SO,팩스용지(15mX216mm/한솔),...,2100,50:34.9,U,NaN,NaN,NaN,오피스디포,윤갑석,NaN,팩스용지 15mx216mm 한솔
4,5,104484,104488,100005,Y,공유,Y,전송완료,SO,팩스용지(30mX210mm/한솔),...,3600,50:34.9,U,NaN,NaN,NaN,오피스디포,윤갑석,NaN,팩스용지 30mx210mm 한솔
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104483,104484,5,104488,204575,Y,공유,Y,전송완료,SZ,BKA-001 무릎 보호대 K,...,0,25:45.8,NaN,cmarket_empty.png,NaN,NaN,NaN,백제현,NaN,bka-001 무릎 보호대 k
104484,104485,4,104488,204574,Y,공유,Y,전송완료,SZ,BKH-002 손목 보호대,...,0,25:45.8,NaN,cmarket_empty.png,NaN,NaN,NaN,백제현,NaN,bkh-002 손목 보호대
104485,104486,3,104488,204585,Y,공유,Y,전송완료,SZ,[고려에이스] Centrifuge Tube Rack 스텐원심관랙,...,0,31:18.6,NaN,cmarket_empty.png,NaN,NaN,고려에이스,박예린,NaN,고려에이스 centrifuge tube rack 스텐원심관랙
104486,104487,2,104488,204587,Y,공유,Y,전송완료,SZ,Bambu 저온용 재사용 가능 스풀,...,0,52:19.6,NaN,cmarket_empty.png,NaN,NaN,뱀부랩,안재용,NaN,bambu 저온용 재사용 가능 스풀


In [11]:
df_test2.columns

Index(['NO', 'ROWNUM', 'totalCount', 'pdt_cd', 'pdt_share', 'pdt_share_txt',
       'send_chk', 'send_chk_txt', 'pdt_code', 'pdt_name', 'pdt_spec',
       'pdt_cas', 'pdt_price', 'pdt_request_date', 'pdt_simg_type', 'pdt_simg',
       'pdt_simg_org', 'pdt_simg_size', 'brand_name', 'dam_name',
       'pdt_simg_file_key', 'pdt_name_clean'],
      dtype='object')

In [12]:
df_new = df_test2[['pdt_name','pdt_name_clean','brand_name','pdt_price','pdt_cas']]

df_new['tokens'] = df_new['pdt_name_clean'].apply(lambda x: x.split()) # 토큰화
df_new['tokens_str'] = df_new['tokens'].apply(lambda x: ' '.join(x)) # 문자열 합침

# pdt_cas가 NaN인 행만 추출
df_nan_cas = df_new[df_new['pdt_cas'].isna()]

# 결과 확인
df_nan_cas
# pdt_cas 가 NaN인 것들을 뽑아내면 시약과 분리되어 데이터가 더 가벼워짐

/var/folders/vh/v6yfwz_x4xz31663ql80v00r0000gq/T/ipykernel_42311/2355649042.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['tokens'] = df_new['pdt_name_clean'].apply(lambda x: x.split()) # 토큰화
/var/folders/vh/v6yfwz_x4xz31663ql80v00r0000gq/T/ipykernel_42311/2355649042.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['tokens_str'] = df_new['tokens'].apply(lambda x: ' '.join(x)) # 문자열 합침


,pdt_name,pdt_name_clean,brand_name,pdt_price,pdt_cas,tokens,tokens_str
0,전산기록지(80컬럼/양미싱),전산기록지 80컬럼 양미싱,오피스디포,25600,NaN,"[전산기록지, 80컬럼, 양미싱]",전산기록지 80컬럼 양미싱
1,전산기록지(132컬럼/양미싱),전산기록지 132컬럼 양미싱,오피스디포,36100,NaN,"[전산기록지, 132컬럼, 양미싱]",전산기록지 132컬럼 양미싱
2,팩스용지(15mX210mm/한솔),팩스용지 15mx210mm 한솔,오피스디포,2100,NaN,"[팩스용지, 15mx210mm, 한솔]",팩스용지 15mx210mm 한솔
3,팩스용지(15mX216mm/한솔),팩스용지 15mx216mm 한솔,오피스디포,2100,NaN,"[팩스용지, 15mx216mm, 한솔]",팩스용지 15mx216mm 한솔
4,팩스용지(30mX210mm/한솔),팩스용지 30mx210mm 한솔,오피스디포,3600,NaN,"[팩스용지, 30mx210mm, 한솔]",팩스용지 30mx210mm 한솔
...,...,...,...,...,...,...,...
104483,BKA-001 무릎 보호대 K,bka-001 무릎 보호대 k,NaN,0,NaN,"[bka-001, 무릎, 보호대, k]",bka-001 무릎 보호대 k
104484,BKH-002 손목 보호대,bkh-002 손목 보호대,NaN,0,NaN,"[bkh-002, 손목, 보호대]",bkh-002 손목 보호대
104485,[고려에이스] Centrifuge Tube Rack 스텐원심관랙,고려에이스 centrifuge tube rack 스텐원심관랙,고려에이스,0,NaN,"[고려에이스, centrifuge, tube, rack, 스텐원심관랙]",고려에이스 centrifuge tube rack 스텐원심관랙
104486,Bambu 저온용 재사용 가능 스풀,bambu 저온용 재사용 가능 스풀,뱀부랩,0,NaN,"[bambu, 저온용, 재사용, 가능, 스풀]",bambu 저온용 재사용 가능 스풀


In [13]:
# minibatchkmeans로 1차 그루핑 + dbscan으로 2차 그루핑 하는 방법

# import pandas as pd
# import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.cluster import MiniBatchKMeans, DBSCAN

# # 데이터 준비
# df_cluster = df.copy()  # 원본 영향 안 주기
# df_cluster['pdt_name_clean'] = df_cluster['pdt_name'].str.lower()  # 소문자
# df_cluster['pdt_name_clean'] = df_cluster['pdt_name_clean'].str.replace(r'[()\[\]/\\]', ' ', regex=True)  # 괄호/슬래시 제거
# df_cluster['pdt_name_clean'] = df_cluster['pdt_name_clean'].str.replace(r'\s+', ' ', regex=True).str.strip()  # 공백 정리

# # pdt_name_clean 컬럼 결측값 처리
# df_new['pdt_name_clean'] = df_new['pdt_name_clean'].fillna('')

# # TF-IDF 벡터화
# vectorizer = TfidfVectorizer(analyzer='word')
# X = vectorizer.fit_transform(df_new['pdt_name_clean'])


# # MiniBatchKMeans로 1차 그룹
# n_clusters = 3000  # 거칠게 쪼개는 수준
# kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, batch_size=1000)
# df_cluster['kmeans_group'] = kmeans.fit_predict(X)

# # DBSCAN 2차 그룹 (각 KMeans 그룹 내) 세밀하게
# final_groups = np.zeros(len(df_cluster), dtype=int)
# current_label = 0

# for grp in df_cluster['kmeans_group'].unique():
#     idx = df_cluster.index[df_cluster['kmeans_group'] == grp]
#     X_sub = X[idx]
    
#     # DBSCAN 적용
#     db = DBSCAN(eps=0.3, min_samples=2, metric='cosine')
#     labels_sub = db.fit_predict(X_sub)
    
#     # -1 (noise) 제외하고, 전체 레이블 offset
#     labels_sub = np.where(labels_sub == -1, -1, labels_sub + current_label)
    
#     final_groups[idx] = labels_sub.max() if labels_sub.max() > 0 else current_label

# df_cluster['final_group'] = final_groups

# #결과 확인
# print(df_cluster[['pdt_name', 'pdt_name_clean', 'kmeans_group', 'final_group']].head(20))


In [ ]:
# gpt 코드 개선 2 노이즈 추가 버전
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import hdbscan

# 원본 영향 없이 처리
df_cluster1 = df_nan_cas.copy()

# 1️⃣ 제품명 전처리
df_cluster1 = df_cluster1.reset_index(drop=True)
df_cluster1['pdt_name_clean'] = df_cluster1['pdt_name_clean'].fillna('')

# 2️⃣ SentenceTransformer 임베딩 (batch 처리)
model = SentenceTransformer('all-MiniLM-L6-v2')
batch_size = 1024
embeddings = []

for i in range(0, len(df_cluster1), batch_size):
    batch_texts = df_cluster1['pdt_name_clean'].iloc[i:i+batch_size].tolist()
    batch_emb = model.encode(batch_texts)
    embeddings.append(batch_emb)

embeddings = np.vstack(embeddings)

# 3️⃣ HDBSCAN 군집화
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=5,
    min_samples=2,
    metric='euclidean',
    cluster_selection_method='eom'
)

cluster_labels = clusterer.fit_predict(embeddings)
df_cluster1['group_id'] = cluster_labels

# 4️⃣ 노이즈(-1) 처리: 가장 가까운 클러스터에 합치기
unique_clusters = [c for c in np.unique(cluster_labels) if c != -1]
cluster_centers = {}

for c in unique_clusters:
    idx = np.where(cluster_labels == c)[0]
    cluster_centers[c] = embeddings[idx].mean(axis=0)

for i, label in enumerate(cluster_labels):
    if label == -1:
        # 노이즈 제품
        distances = [np.linalg.norm(embeddings[i] - cluster_centers[c]) for c in unique_clusters]
        nearest_cluster = unique_clusters[np.argmin(distances)]
        df_cluster1.at[i, 'group_id'] = nearest_cluster

# 5️⃣ 각 클러스터 대표 제품 선정
representatives = []
for grp in df_cluster1['group_id'].unique():
    idx = np.where(df_cluster1['group_id'] == grp)[0]
    group_embeddings = embeddings[idx]
    center = group_embeddings.mean(axis=0)
    distances = np.linalg.norm(group_embeddings - center, axis=1)
    rep_idx = idx[np.argmin(distances)]
    representatives.append((grp, df_cluster1.iloc[rep_idx]['pdt_name']))

df_representatives = pd.DataFrame(representatives, columns=['group_id', '대표_제품'])

# 6️⃣ 결과 확인
print(df_cluster1[['pdt_name', 'pdt_name_clean', 'group_id']].head(20))
print(df_representatives.head(20))

# 7️⃣ CSV 저장
# df_cluster1.to_csv('df_cluster1_final.csv', index=False)


In [17]:
# print(len(df_cluster1), embeddings.shape[0]) # df_new와 섞여서 값이 튀었던 것 같음. 제거 이후 다시 코드 돌리니 같은 값이 나오는 것을 확인.


In [16]:
ct_data = pd.read_csv('/Users/c-market/Downloads/df_cluster1.csv')
ct_data

,pdt_name,pdt_name_clean,brand_name,pdt_price,pdt_cas,tokens,tokens_str,group_id
0,전산기록지(80컬럼/양미싱),전산기록지 80컬럼 양미싱,오피스디포,25600,NaN,"['전산기록지', '80컬럼', '양미싱']",전산기록지 80컬럼 양미싱,2337
1,전산기록지(132컬럼/양미싱),전산기록지 132컬럼 양미싱,오피스디포,36100,NaN,"['전산기록지', '132컬럼', '양미싱']",전산기록지 132컬럼 양미싱,2365
2,팩스용지(15mX210mm/한솔),팩스용지 15mx210mm 한솔,오피스디포,2100,NaN,"['팩스용지', '15mx210mm', '한솔']",팩스용지 15mx210mm 한솔,1988
3,팩스용지(15mX216mm/한솔),팩스용지 15mx216mm 한솔,오피스디포,2100,NaN,"['팩스용지', '15mx216mm', '한솔']",팩스용지 15mx216mm 한솔,1988
4,팩스용지(30mX210mm/한솔),팩스용지 30mx210mm 한솔,오피스디포,3600,NaN,"['팩스용지', '30mx210mm', '한솔']",팩스용지 30mx210mm 한솔,1987
...,...,...,...,...,...,...,...,...
63929,BKA-001 무릎 보호대 K,bka-001 무릎 보호대 k,NaN,0,NaN,"['bka-001', '무릎', '보호대', 'k']",bka-001 무릎 보호대 k,1165
63930,BKH-002 손목 보호대,bkh-002 손목 보호대,NaN,0,NaN,"['bkh-002', '손목', '보호대']",bkh-002 손목 보호대,1545
63931,[고려에이스] Centrifuge Tube Rack 스텐원심관랙,고려에이스 centrifuge tube rack 스텐원심관랙,고려에이스,0,NaN,"['고려에이스', 'centrifuge', 'tube', 'rack', '스텐원심관랙']",고려에이스 centrifuge tube rack 스텐원심관랙,871
63932,Bambu 저온용 재사용 가능 스풀,bambu 저온용 재사용 가능 스풀,뱀부랩,0,NaN,"['bambu', '저온용', '재사용', '가능', '스풀']",bambu 저온용 재사용 가능 스풀,1012


### 그룹핑 된 아이템들을 확인한 결과 머신러닝을 기반으로 진행했을 때 완벽히 일치하는 제품들로 뽑아내기 어려움 선제적인 전처리 or 이후 분류할 수 있는 방법이 있으면 좋지 않을까